In [71]:
from PIL import Image, ImageOps
import numpy as np
import math

In [72]:
def bytestring_to_int(string: str):
    """
    Convert the input binary string into it's integer equivalent
    Eg: 00000011 -> 3
    """

    val = 0
    power_of_twos = [128, 64, 32, 16, 8, 4, 2, 1]

    for i in range(len(string)):
        if string[i] == "1":
            val += power_of_twos[i]

    return val

def int_to_bytestring(val: int):
    """
    Convert the input integer value into it's bit string equivalent
    Eg: 3 -> 00000011
    """

    bytestring = ""
    power_of_twos = [128, 64, 32, 16, 8, 4, 2, 1]

    for power in power_of_twos:
        if val >= power:
            bytestring += '1'
            val -= power
        else:
            bytestring += '0'

    return bytestring

# Fibonacci Encoding

## Compressed file header

Compressed file contains a header which contains the rank of pixel frequencies. This is required while decompressing the file.

The header format used in this case is as follows:
- First byte contains the number of unique pixel values in the image (`n`). Since each pixel can have a value between 0-255, this number can fit within 1 byte.
- Next `n` bytes will contain the actual pixel value sorted in descending order of their frequency.

As a result, the header contains `n+1` bytes. All bytes following this consist of the actual file data that needs to be decompressed.

In [73]:
def get_max_fibo(n):
    # Get max fibonacci number less than equal to n
    # Returns (fibo_num, count)
    # count returns the 0-based index of fibo_num in the Fibonacci sequence
    a = 1
    prev = 0
    count = 0
    while a <= n:
        count += 1
        new = a + prev
        if new > n:
            break
        prev = a
        a = new

    return a, count - 2

def fibonacci_code(n):
    # Returns Fibonacci code for n
    indices = []
    while n > 0:
        fibo, count = get_max_fibo(n)
        n = n - fibo
        indices.append(count)

    code = ""
    for i in range(indices[0] + 1):
        if i in indices:
            code += "1"
        else:
            code += "0"

    return code + "1"

In [131]:
class FibonacciCode:
    def __init__(self):
        self.compute_fibonacci_codes()

    def compute_fibonacci_codes(self):
        # Stores fibonacci codes for 1,2,...,256 in indices 0,1,...,255
        self.fibonacci_codes = [fibonacci_code(i+1) for i in range(256)]

    def open_grayscale(self, path: str):
        img = Image.open(path)
        self.img = ImageOps.grayscale(img)

    def _compute_pixel_frequencies(self):
        pixel_freq_map = {}
        pixels = np.asarray(self.img)
        for row in pixels:
            for pixel in row:
                pixel_frequency = pixel_freq_map.get(pixel, None)
                if pixel_frequency:
                    pixel_freq_map[pixel] = pixel_frequency + 1
                else:
                    pixel_freq_map[pixel] = 1
        return self._map_to_tuple(pixel_freq_map)

    def _map_to_tuple(self, pixel_freq_map):
        tuple_list = []     # Tuple: (pixel, frequency)
        for pixel in pixel_freq_map:
            tuple_list.append((pixel, pixel_freq_map[pixel]))
        tuple_list.sort(reverse=True, key=lambda x: x[1])
        return tuple_list

    def _create_file_heading(self, pixel_freq_tuple):
        heading = [len(pixel_freq_tuple)]
        for pixel, freq in pixel_freq_tuple:
            heading.append(int(pixel))
        return heading

    def _get_bytes_arr(self, data: str):
        bytes_arr = []
        for i in range(0, len(data), 8):
            byte_val = bytestring_to_int(data[i:i+8])
            bytes_arr.append(byte_val)
        return bytes_arr

    def _write_to_file(self, path: str, bytes_arr: list[int]):
        output_file_path = path.split(".")[0] + "_compressed.bin"
        with open(output_file_path, "wb") as f:
            for byte in bytes_arr:
                f.write(byte.to_bytes(1, "big"))

    def encode(self, path: str):
        self.open_grayscale(path)
        pixel_frequencies = self._compute_pixel_frequencies()
        encoded_bytes_arr = self._create_file_heading(pixel_frequencies)
        encoded_data = ""

        pixels = np.asarray(self.img)
        for row in pixels:
            for pixel in row:
                for idx, tuple in enumerate(pixel_frequencies):
                    pxl, freq = tuple
                    if pxl == pixel:
                        encoded_data += self.fibonacci_codes[idx]


        for val in self._get_bytes_arr(encoded_data):
            encoded_bytes_arr.append(val)

        self._write_to_file(path, encoded_bytes_arr)

    def decode(self, path: str):
        byte_val_arr = []
        with open(path, "rb") as f:
            contents = f.read()
            byte_val_arr = list(contents)

        n = byte_val_arr[0]
        sorted_pixel_values = []
        for i in range(1, n + 1):
            sorted_pixel_values.append(byte_val_arr[i])

        encoded_data = ""
        for i in range(n + 1, len(byte_val_arr)):
            encoded_data += int_to_bytestring(byte_val_arr[i])

        curr = ""
        prev = ""
        pixels = []
        for char in encoded_data:
            curr += char
            if prev + char == "11":
                idx = self.fibonacci_codes.index(curr)
                pixels.append(sorted_pixel_values[idx])
                curr = ""
                prev = ""
            else:
                prev = char

        computed_height = computed_width = int(math.sqrt(len(pixels)))
        image_data = np.array(pixels).reshape((computed_height, -1))

        new_image = Image.fromarray(np.uint8(image_data), "L")
        new_image.show()
        new_image.save('decompressed.bmp')



In [128]:
files = [
    "data/gs_50X50.bmp",
    "data/gs_100X100.bmp",
    "data/gs_200X200.bmp",
    "data/gs_300X300.bmp",
    "data/gs_400X400.bmp",
    "data/gs_500X500.bmp",
    ]

In [132]:
selected_image = files[0]
fibo_encoder = FibonacciCode()
fibo_encoder.encode(selected_image)

In [133]:
compressed_file = selected_image.split(".")[0] + "_compressed.bin"
fibo_encoder.decode(compressed_file)